In [ ]:
import requests
import json

apiKey = "E3ECE458BA26350EAF264840A63BF51E"
steamId = "76561198345197403"
ownedGameApi = f"http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={apiKey}&steamid={steamId}&format=json&include_appinfo=True"
friendListApi = f"http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={apiKey}&steamid={steamId}&relationship=friend"
recentPlayedApi = f"http://api.steampowered.com/IPlayerService/GetRecentlyPlayedGames/v0001/?key={apiKey}&steamid={steamId}&format=json"

user_dict = {'userid':steamId, 'recentPlayedGames':[], 'ownedGames' : [], 'friendList': []}

response = requests.get(recentPlayedApi).json()
games = response['response']['games']
for game in games:
  user_dict['recentPlayedGames'].append((game['name'],game['playtime_forever']))

response = requests.get(ownedGameApi).json()
games = response['response']['games']
for game in games:
  user_dict['ownedGames'].append((game['name'],game['playtime_forever']))

try:
  response =  requests.get(friendListApi).json()
  friends = response['friendslist']['friends']
  for friend in friends:
    user_dict['friendList'].append(friend['steamid'])
except:
  print("Could not retrieve friends data")

user_dict['ownedGames'].sort(key = lambda x: x[1], reverse=True)
print(user_dict)

{'userid': '76561198345197403', 'recentPlayedGames': [('Moncage', 23)], 'ownedGames': [('Monster Hunter: World', 19228), ('The Binding of Isaac: Rebirth', 14102), ('Terraria', 13027), ('ELDEN RING', 11524), ('Fall Guys', 3541), ('Bloodstained: Ritual of the Night', 2434), ('V Rising', 2431), ('Richman10', 2350), ('Human: Fall Flat', 1934), ('The Witness', 1804), ('Hades', 1798), ('Salt and Sanctuary', 1633), ('Yu-Gi-Oh! Legacy of the Duelist', 1575), ('Resident Evil 6', 1371), ('Portal 2', 1319), ('Command and Conquer: Red Alert 3', 1179), ('The Forest', 1021), ('Slay the Spire', 1019), ('Blasphemous', 989), ('Draw & Guess', 820), ('Red Dead Redemption 2', 716), ('The Escapists 2', 708), ("Don't Starve Together", 670), ('Dead by Daylight', 603), ('PUBG: BATTLEGROUNDS', 538), ('Richman 11', 503), ('Fate/EXTELLA', 489), ('TerraTech', 483), ('Escape Simulator', 460), ('Castle Crashers', 451), ('Nobody Saves the World', 422), ('Trine 4: The Nightmare Prince', 411), ('Inside the Backrooms',

In [ ]:
users = []

def getFriends(friendId, level):
    if level > 7:
      return
   
    friendListApi = f"http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={apiKey}&steamid={friendId}&relationship=friend"
    print(friendListApi)
    #try:
    response = requests.get(friendListApi).json()
    print(response)
    friends = response['friendslist']['friends']
    for friend in friends:
      users.append(friend['steamid'])
      getFriends(friend['steamid'], level+1)    
    #except:
    #  print("Could not retrieve friends data")
    print(level)
    #level+=1

In [ ]:
for friendId in user_dict['friendList']:
  getFriends(friendId, 0)
print(users)

In [ ]:
print(user_dict['friendList'])

['76561198110161332', '76561198156029203', '76561198204064957', '76561198220572464', '76561198300874429', '76561198319387925', '76561198344269123', '76561198368915478', '76561198412582160', '76561198417834212', '76561198447129756']


In [ ]:
with open('user_data.json', 'w', encoding='utf-8') as f:
  json.dump(user_dict, f, ensure_ascii=False, indent=4)

In [2]:
from bs4 import BeautifulSoup
import requests, time
import json

#data_dict = {'name':[], 'summary':[], 'date':[], 'platform': [], 'score':[], 'url':[], 'userscore':[], 'genre':[]} # Data Structure
data_dict = {'name':[], 'summary':[], 'date':[], 'platform': [], 'score':[], 'url':[], 'userscore':[]}


urls = {
          "Switch" : "https://www.metacritic.com/browse/games/release-date/available/switch/metascore",
          "PC": "https://www.metacritic.com/browse/games/release-date/available/pc/metascore",
          "PS" : "https://www.metacritic.com/browse/games/release-date/available/ps5/metascore"
}

In [1]:
def parse_url(url):
    userAgent = {'User-agent': 'Mozilla/5.0'}
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    source = requests.get(url, headers=headers).text
    soup = BeautifulSoup(source, 'html.parser')  
    content = soup.find_all('table')
    tblnum = 0
    while tblnum < len(content):
        table_rows = content[tblnum].find_all('tr')
        for tr in table_rows:
            td = tr.find_all('td')
            if td!=[]:
                for a in td[1].find_all('a', {"class":"title"}):
                    data_dict['name'].append(a.find('h3').text)
                for date in td[1].find_all('span',{"class":""}):
                    data_dict['date'].append(date.text)
                for platform in td[1].find_all('span',{"class":"data"}):
                    data_dict['platform'].append(platform.text.strip())
                scores =  td[1].find_all('div',{"class":"metascore_w"})
                summary = td[1].find_all('div',{"class":"summary"})
                data_dict['score'].append(scores[0].text.strip())
                data_dict['userscore'].append(scores[2].text.strip())
                #print(summary)
                data_dict['summary'].append(summary[0].text.strip())
                for a in td[1].find_all('a', {"class":"title"} ,href=True):
                    data_dict['url'].append(a['href'])

        tblnum += 1

def parsePage():
    baseUrl = "https://www.metacritic.com"
    for url in data_dict['url']:
        fullUrl = baseUrl + url
        print(fullUrl)
        userAgent = {'User-agent': 'Mozilla/5.0'}
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
        try: 
            source = requests.get(fullUrl, headers=userAgent).text
            soup = BeautifulSoup(source, 'html.parser')  
            content = soup.find_all('li', {"class" : "summary_detail product_genre"})
            genre_str = ""
            for item in content:  
                genres = item.find_all('span', {"class" : "data"})
                for genre in genres:
                    if genre == genres[-1]:
                        genre_str += genre.text.strip() 
                    else:
                        genre_str += genre.text.strip() + ","
            data_dict['genre'].append(genre_str)

            #content = soup.find_all('li', {"class" : "summary_detail product_summary"})
            #spans = content[0].find_all('span', {"class" : "blurb blurb_expanded"})
            #data_dict['summary'].append(spans[0].text)    
        except:
           print("Coult not open url")
           data_dict['summary'].append("No content")    

        time.sleep(5)

In [3]:
def scraper(platform, page=10):
  url = urls[platform]
  for i in range(page):
    fullUrl = url + "?page=" + str(i)
    parse_url(fullUrl)

  #parsePage()

In [4]:
scraper("PC", 10)

In [5]:
scraper("Switch", 5)

In [6]:
scraper("PS", 5)

In [7]:
for i in range(len(data_dict['summary'])):
  data_dict['summary'][i] = data_dict['summary'][i].strip()

with open('data.json', 'w', encoding='utf-8') as f:
  json.dump(data_dict, f, ensure_ascii=False, indent=4)

In [8]:
import pandas as pd
df = pd.read_json (r'data.json')
df

,name,summary,date,platform,score,url,userscore
0,Disco Elysium: The Final Cut,Disco Elysium - The Final Cut is the definitiv...,2021-03-30,PC,97,/game/pc/disco-elysium-the-final-cut,8.3
1,Half-Life 2,[Metacritic's 2004 PC Game of the Year] By ta...,2004-11-16,PC,96,/game/pc/half-life-2,9.2
2,Grand Theft Auto V,Los Santos: a sprawling sun-soaked metropolis ...,2015-04-13,PC,96,/game/pc/grand-theft-auto-v,7.8
3,Out of the Park Baseball 2007,[Metacritic's 2007 PC Game of the Year] OOTP ...,2006-03-23,PC,96,/game/pc/out-of-the-park-baseball-2007,2.6
4,The Orange Box,Games included in The Orange Box compilation: ...,2007-10-10,PC,96,/game/pc/the-orange-box,9.1
...,...,...,...,...,...,...,...
1948,Praey for the Gods,Praey for the Gods is a boss climbing open wor...,2021-12-14,PlayStation 5,46,/game/playstation-5/praey-for-the-gods,7.0
1949,Evil Inside,Evil Inside is a first-person psychological ho...,2021-03-25,PlayStation 5,43,/game/playstation-5/evil-inside,5.1
1950,Babylon's Fall,Experience acclaimed developer PlatinumGames' ...,2022-03-03,PlayStation 5,41,/game/playstation-5/babylons-fall,2.1
1951,eFootball 2022,"From ""PES"" to ""eFootball™ It all began with th...",2021-09-30,PlayStation 5,34,/game/playstation-5/efootball-2022,1.2


In [9]:
df[df['platform'] == "PlayStation 5"]

,name,summary,date,platform,score,url,userscore
1500,Elden Ring,A New World Created By Hidetaka Miyazaki And G...,2022-02-25,PlayStation 5,96,/game/playstation-5/elden-ring,7.7
1501,The Witcher 3: Wild Hunt - Complete Edition,"You are Geralt of Rivia, mercenary monster sla...",2022-12-14,PlayStation 5,94,/game/playstation-5/the-witcher-3-wild-hunt---...,7.8
1502,God of War: Ragnarok,The freezing winds of Fimbulwinter have come t...,2022-11-09,PlayStation 5,94,/game/playstation-5/god-of-war-ragnarok,7.9
1503,Hades,Defy the god of the dead as you hack and slash...,2021-08-13,PlayStation 5,93,/game/playstation-5/hades,8.6
1504,Demon's Souls,From PlayStation Studios and Bluepoint Games c...,2020-11-11,PlayStation 5,92,/game/playstation-5/demons-souls,8.0
...,...,...,...,...,...,...,...
1948,Praey for the Gods,Praey for the Gods is a boss climbing open wor...,2021-12-14,PlayStation 5,46,/game/playstation-5/praey-for-the-gods,7.0
1949,Evil Inside,Evil Inside is a first-person psychological ho...,2021-03-25,PlayStation 5,43,/game/playstation-5/evil-inside,5.1
1950,Babylon's Fall,Experience acclaimed developer PlatinumGames' ...,2022-03-03,PlayStation 5,41,/game/playstation-5/babylons-fall,2.1
1951,eFootball 2022,"From ""PES"" to ""eFootball™ It all began with th...",2021-09-30,PlayStation 5,34,/game/playstation-5/efootball-2022,1.2


In [10]:
import sqlite3

In [11]:
conn = sqlite3.connect('games.db')

In [12]:
df.to_sql('games', conn, if_exists='replace')